In [ ]:
import tkinter as tk
import tkinter.messagebox as messagebox
import math

class GameAI:  # Abstract base class for AI (you'll create specific AI classes later)
    def get_best_move(self, board):
        raise NotImplementedError("get_best_move() must be implemented in subclasses")


class MinimaxAI(GameAI):
    def __init__(self, depth):
        self.depth = depth

    def get_best_move(self, board):
        alpha = -math.inf
        beta = math.inf
        best_move = None
        best_value = -math.inf

        for move in self.get_possible_moves(board, "X"): # "X" is the AI's marker
            new_board = self.make_move(board, move, "X")
            value = self.minimax(new_board, self.depth, alpha, beta, False) # AI is maximizing player
            if value > best_value:
                best_value = value
                best_move = move
        return best_move

    def minimax(self, board, depth, alpha, beta, maximizingPlayer):
        if depth == 0 or self.is_terminal_node(board):
            return self.evaluate(board)

        if maximizingPlayer:
            maxEval = -math.inf
            for move in self.get_possible_moves(board, "X"):
                new_board = self.make_move(board, move, "X")
                eval = self.minimax(new_board, depth - 1, alpha, beta, False)
                maxEval = max(maxEval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return maxEval
        else:
            minEval = math.inf
            for move in self.get_possible_moves(board, "O"): # "O" is the opponent's marker
                new_board = self.make_move(board, move, "O")
                eval = self.minimax(new_board, depth - 1, alpha, beta, True)
                minEval = min(minEval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return minEval

    def is_terminal_node(self, board):
        return self.check_winner(board) is not None or self.is_board_full(board)

    def evaluate(self, board):
        winner = self.check_winner(board)
        if winner == "X":
            return 10
        elif winner == "O":
            return -10
        else:
            return 0

    def get_possible_moves(self, board, player):
        moves = []
        for row in range(3):
            for col in range(3):
                if board[row][col] == " ":
                    moves.append((row, col))
        return moves

    def make_move(self, board, move, player):
        row, col = move
        new_board = [row[:] for row in board] # Create a copy of the board
        new_board[row][col] = player
        return new_board

    def check_winner(self, board):
        for player in ["X", "O"]:
            # Check rows
            for row in range(3):
                if all(board[row][col] == player for col in range(3)):
                    return player
            # Check columns
            for col in range(3):
                if all(board[row][col] == player for row in range(3)):
                    return player
            # Check diagonals
            if all(board[i][i] == player for i in range(3)) or \
               all(board[i][2 - i] == player for i in range(3)):
                return player
        return None

    def is_board_full(self, board):
        return all(all(cell != " " for cell in row) for row in board)



class TicTacToeGUI:
    def __init__(self, master, ai):  # Add ai as parameter
        self.master = master
        master.title("Tic-Tac-Toe")
        self.ai = ai # Store the AI object

        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.buttons = {}
        self.create_board()

        self.current_player = "X"  # X starts
        self.game_over = False

    def create_board(self):
        for row in range(3):
            for col in range(3):
                button = tk.Button(
                    self.master,
                    text=" ",
                    width=10,
                    height=5,
                    command=lambda r=row, c=col: self.square_clicked(r, c),
                )
                button.grid(row=row, column=col)
                self.buttons[(row, col)] = button

    def square_clicked(self, row, col):
        if self.game_over or self.board[row][col] != " ":
            return  # Ignore clicks if game is over or square is occupied

        self.board[row][col] = self.current_player
        self.buttons[(row, col)].config(text=self.current_player)

        winner = self.check_winner()
        if winner:
            self.game_over = True
            messagebox.showinfo("Game Over", f"{winner} wins!")
            self.disable_buttons()
            return

        if self.is_board_full():
            self.game_over = True
            messagebox.showinfo("Game Over", "It's a tie!")
            self.disable_buttons()
            return

        self.switch_player()

        if self.current_player == "O" and isinstance(self.ai, MinimaxAI): # AI's turn
            self.master.after(200, self.ai_move) # Small delay for AI move


    def ai_move(self):
        best_move = self.ai.get_best_move(self.board)
        if best_move:
           row, col = best_move
           self.square_clicked(row, col)


    def switch_player(self):
        self.current_player = "O" if self.current_player == "X" else "X"

    def check_winner(self):
        for player in ["X", "O"]:
            # Check rows
            for row in range(3):
                if all(self.board[row][col] == player for col in range(3)):
                    return player
            # Check columns
            for col in range(3):
                if all(self.board[row][col] == player for row in range(3)):
                    return player
            # Check diagonals
            if all(self.board[i][i] == player for i in range(3)) or \
               all(self.board[i][2 - i] == player for i in range(3)):
                return player
        return None

    def is_board_full(self):
        return all(all(cell != " " for cell in row) for row in self.board)

    def disable_buttons(self):
        for button in self.buttons.values():
            button.config(state=tk.DISABLED)


root = tk.Tk()
ai = MinimaxAI(depth=3) # Create AI with depth 3
game = TicTacToeGUI(root, ai) # Pass ai to the game
root.mainloop()